In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
from deeprte.config import make_config
config = make_config()

In [3]:
config

best_model_eval_metric: ''
best_model_eval_metric_higher_is_better: true
checkpoint_interval_type: null
dataset:
  buffer_size: 5000
  data_path: /workspaces/deeprte/rte_data/rte_data/matlab/eval-data/test_shape.mat
  data_split:
    is_split_datasets: true
    num_test_samples: 2
    num_train_samples: 6
    num_val_samples: 2
    save_path: /workspaces/deeprte/rte_data/rte_data/matlab/eval-data/test_ds.npz
    train_validation_split_rate: 0.8
  max_intra_op_parallelism: 1
  num_samples: 10
  pre_shuffle: true
  pre_shuffle_seed: 42
  threadpool_size: 48
  train:
    batch_size: 6
    collocation_sizes: 500
    repeat: 1
  validation:
    batch_size: 2
eval_initial_weights: false
experiment_kwargs: {}
interval_type: secs
log_all_train_data: false
log_tensors_interval: 60
log_train_data_interval: 120.0
logging_interval_type: null
max_checkpoints_to_keep: 5
one_off_evaluate: false
random_mode_eval: same_host_same_device
random_mode_train: unique_host_unique_device
random_seed: 42
save_c

In [4]:
data_path = "/workspaces/deeprte/rte_data/rte_data/matlab/eval-data/test_shape.mat"

from deeprte.data.pipeline import DataPipeline
data_pipeline = DataPipeline(data_path)
data = data_pipeline.process(pre_shuffle=True, is_split_test_samples=True, num_test_samples= 2)

from deeprte.model.tf.rte_dataset import np_to_tensor_dict

tf_data = np_to_tensor_dict(data)

from deeprte.model.tf.rte_dataset import divide_batch_feat
batched_data, unbatched_data = divide_batch_feat(tf_data)

2022-11-29 08:24:30.233937: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 08:24:30.369339: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [5]:
import tensorflow as tf

tf.nest.map_structure(lambda x:x.shape, tf_data)

{'sigma': TensorShape([8, 1681, 2]),
 'psi_label': TensorShape([8, 40344]),
 'scattering_kernel': TensorShape([8, 40344, 24]),
 'boundary': TensorShape([8, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [6]:
from deeprte.model.tf.input_pipeline import split_feat,process_features, make_device_batch
import jax
train_feat = split_feat(tf_data, 0.8, True)
# ds, unbatched_feat = load_and_split_data(tf_data, data_config, True)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tf.nest.map_structure(lambda x:x.shape, train_feat)

{'sigma': TensorShape([6, 1681, 2]),
 'psi_label': TensorShape([6, 40344]),
 'scattering_kernel': TensorShape([6, 40344, 24]),
 'boundary': TensorShape([6, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [8]:
data_config = config.dataset

In [9]:
train_ds = process_features(train_feat, is_training=True, batch_sizes= make_device_batch(data_config.train.batch_size, jax.device_count()),collocation_sizes=data_config.train.collocation_sizes)
# train_ds = process_features(train_feat, is_training=True, batch_sizes= [2,3],collocation_sizes=data_config.train.collocation_sizes)

In [10]:
tf.nest.map_structure(lambda x:x.shape, next(train_ds))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


{'sigma': (1, 6, 1681, 2),
 'psi_label': (1, 6, 500),
 'scattering_kernel': (1, 6, 500, 24),
 'boundary': (1, 6, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 500, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [11]:
val_feat = split_feat(tf_data, 0.8, False)

In [12]:
val_ds = process_features(val_feat, is_training=False, batch_sizes= make_device_batch(data_config.validation.batch_size, jax.device_count()))

In [13]:
tf.nest.map_structure(lambda x:x.shape, next(val_ds))

{'sigma': (1, 2, 1681, 2),
 'psi_label': (1, 2, 40344),
 'scattering_kernel': (1, 2, 40344, 24),
 'boundary': (1, 2, 1968),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 40344, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [14]:
from deeprte.model.tf.input_pipeline import load_tf_data, tf_data_to_generator

In [20]:
ds = load_tf_data(data_path, pre_shuffle=True,is_split_test_samples=True, num_test_samples = 2)

In [21]:
tf.nest.map_structure(lambda x:x.shape, ds)

{'sigma': TensorShape([8, 1681, 2]),
 'psi_label': TensorShape([8, 40344]),
 'scattering_kernel': TensorShape([8, 40344, 24]),
 'boundary': TensorShape([8, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [22]:
tfds = tf_data_to_generator(ds,True, 0.8, [2,3],collocation_sizes=500)

In [23]:
tf.nest.map_structure(lambda x:x.shape, next(tfds))

{'sigma': (2, 3, 1681, 2),
 'psi_label': (2, 3, 500),
 'scattering_kernel': (2, 3, 500, 24),
 'boundary': (2, 3, 1968),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 500, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}